In [89]:
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns
import scipy
import sklearn

In [90]:
dataset=pd.read_csv("/Users/empty/PycharmProjects/Machine-Learning-Project-Group-6/Data/airfoil_self_noise.dat",sep="\t",index_col=False,names=["Frequency","Angle","Chord","Velocity","Suction","Pressure Level"], header = None)
#dataset read the csv and use ; to separate it

In [91]:
from sklearn.model_selection import train_test_split
train, test=train_test_split(dataset,test_size=0.2,random_state=1)

In [93]:
x_train = train.drop("Pressure Level", axis = 1)
y_train = train["Pressure Level"]
x_test = test.drop("Pressure Level", axis = 1)
y_test = test["Pressure Level"]
#Pressure Level don't need to be normalized

In [94]:
from sklearn.preprocessing import Normalizer
norm = Normalizer()
norm.fit(x_train)
n_x_train = norm.transform(x_train)  #normalized train dataset
n_x_test = norm.transform(x_test) #normalized test dataset

In [111]:
from sklearn.feature_selection import SelectKBest, f_classif
selector = SelectKBest(f_classif,k=3)  #降维成3, chi2不可以使用浮点数
s_n_x_train = selector.fit_transform(n_x_train, y_train)
s_n_x_test = selector.transform(n_x_test)

In [130]:
from sklearn.svm import SVR

# svr=SVR(C=1.0,kernel="poly",degree=3,gamma='auto')
# svr=SVR(kernel="poly", degree=3, C=1, epsilon=0.1, gamma="auto")
svr = SVR()
#kernel 核函数，多项式支持向量回归用这个
#最高次方 2
#惩罚因子 1.0, 宽度
#epsilon超参数调节
svr.fit(s_n_x_train,y_train)

SVR()

In [122]:
s_n_x_train

array([[1.00631323e-02, 1.21715981e-04, 5.69445503e-02],
       [1.05668692e-02, 1.60237898e-04, 1.12450414e-01],
       [8.53804901e-03, 2.40962716e-04, 8.77521703e-02],
       ...,
       [1.50790501e-03, 4.03166180e-06, 6.28558297e-03],
       [0.00000000e+00, 1.82821218e-04, 2.53518486e-02],
       [5.93150903e-03, 1.58589820e-05, 4.45175362e-02]])

In [131]:
regression_score=svr.score(s_n_x_test,y_test)
print('The regression accuracy score is {:03.2f}'.format(regression_score))
regression_score
#如果不进行标准化，模型效果会特别差

The regression accuracy score is 0.16


0.15748051753448666

In [132]:
from sklearn.model_selection import cross_val_score
y_predict=svr.predict(s_n_x_test)
cross_val_score(svr, s_n_x_train, y_train, cv=5, scoring='neg_mean_squared_error')

array([-35.16303006, -37.92591314, -31.94268394, -40.34971488,
       -38.17845231])

In [133]:
y_train

701     120.154
1344    131.448
556     131.807
785     121.238
1206    124.166
         ...   
715     116.560
905     125.353
1096    120.324
235     128.805
1061    133.756
Name: Pressure Level, Length: 1202, dtype: float64

In [134]:
x_train

,Frequency,Angle,Chord,Velocity,Suction
701,1250,12.6,0.1524,71.3,0.048316
1344,630,6.7,0.1016,71.3,0.004783
556,630,5.4,0.1524,55.5,0.004333
785,10000,4.2,0.0508,71.3,0.001428
1206,2500,17.4,0.0254,31.7,0.017663
...,...,...,...,...,...
715,800,12.6,0.1524,39.6,0.058411
905,400,15.4,0.0508,39.6,0.028259
1096,6300,9.5,0.0254,39.6,0.004498
235,1250,0.0,0.2286,31.7,0.002724
